In [11]:
from abc import ABC, abstractmethod
from typing import List, Set
import copy
import time
import sys


class Vidnoshennya(ABC):
    def __init__(self, n: int = 0):
        self.n = n

    def __copy__(self):
        return copy.deepcopy(self)  # копіювання об'єкта
    @abstractmethod
    def peretyn(self, other: 'Vidnoshennya') -> 'Vidnoshennya':
        pass
    @abstractmethod
    def obyednannya(self, other: 'Vidnoshennya') -> 'Vidnoshennya':
        pass
    @abstractmethod
    def riznytsya(self, other: 'Vidnoshennya') -> 'Vidnoshennya':
        pass
    @abstractmethod
    def sym_riznytsya(self, other: 'Vidnoshennya') -> 'Vidnoshennya':
        pass
    @abstractmethod
    def dopovnennya(self) -> 'Vidnoshennya':
        pass
    @abstractmethod
    def obernene(self) -> 'Vidnoshennya':
        pass
    @abstractmethod
    def kompozytsiya(self, other: 'Vidnoshennya') -> 'Vidnoshennya':
        pass
    @abstractmethod
    def zvuzhennya(self, subset: List[int]) -> 'Vidnoshennya':
        pass
    @abstractmethod
    def vklyuchennya(self, other: 'Vidnoshennya') -> bool:
        pass
    @abstractmethod
    def dvoiste(self) -> 'Vidnoshennya':
        pass
    @abstractmethod
    def print(self):
        pass


class VidnoshennyaMatr(Vidnoshennya):
    def __init__(self, matrix: List[List[int]] = None, n: int = 0, tipo: str = None):
        if matrix is not None:
            super().__init__(len(matrix))
            self.B = copy.deepcopy(matrix)
        elif tipo:  # конструктор елементарних
            super().__init__(n)
            self.B = [[0] * n for _ in range(n)]
            if tipo == "povne":
                self.B = [[1] * n for _ in range(n)]
            elif tipo == "diag":
                for i in range(n):
                    self.B[i][i] = 1
            elif tipo == "antidiag":
                for i in range(n):
                    for j in range(n):
                        if i != j:
                            self.B[i][j] = 1
        else:
            super().__init__(n)
            self.B = [[0] * n for _ in range(n)]
    def peretyn(self, other):
        return VidnoshennyaMatr([[self.B[i][j] & other.B[i][j] for j in range(self.n)] for i in range(self.n)])
    def obyednannya(self, other):
        return VidnoshennyaMatr([[self.B[i][j] | other.B[i][j] for j in range(self.n)] for i in range(self.n)])
    def riznytsya(self, other):
        return VidnoshennyaMatr([[self.B[i][j] & (1 - other.B[i][j]) for j in range(self.n)] for i in range(self.n)])
    def sym_riznytsya(self, other):
        return VidnoshennyaMatr([[self.B[i][j] ^ other.B[i][j] for j in range(self.n)] for i in range(self.n)])
    def dopovnennya(self):
        return VidnoshennyaMatr([[1 - self.B[i][j] for j in range(self.n)] for i in range(self.n)])
    def obernene(self):
        return VidnoshennyaMatr([[self.B[j][i] for j in range(self.n)] for i in range(self.n)])
    def kompozytsiya(self, other):
        res = [[0] * self.n for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.n):
                for k in range(self.n):
                    if self.B[i][k] and other.B[k][j]:
                        res[i][j] = 1
                        break
        return VidnoshennyaMatr(res)
    def zvuzhennya(self, subset: List[int]):
        res = [[self.B[i][j] for j in subset] for i in subset]
        return VidnoshennyaMatr(res)
    def vklyuchennya(self, other):
        for i in range(self.n):
            for j in range(self.n):
                if self.B[i][j] and not other.B[i][j]:
                    return False
        return True
    def dvoiste(self):
        res = [[0]*self.n for _ in range(self.n)]
        for i in range(self.n):
            for j in range(self.n):
                if i == j:
                    continue
                if self.B[i][j] == 0 and self.B[j][i] == 0:
                    res[i][j] = 1
        for i in range(self.n):
            if self.B[i][i] == 0:
                res[i][i] = 1
        return VidnoshennyaMatr(res)
    def to_zriz(self):
        zr = []
        for x in range(self.n):
            zr_set = set()
            for y in range(self.n):
                if self.B[x][y] == 1:
                    zr_set.add(y)
            zr.append(zr_set)
        return VidnoshennyaZriz(self.n, zr)
    def print(self):
        for row in self.B:
            print(" ".join(map(str, row)))
        print()



class VidnoshennyaZriz(Vidnoshennya):
    def __init__(self, n: int = 0, zrizy: List[Set[int]] = None, tipo: str = None):
        super().__init__(n)
        if zrizy is not None:
            self.R_plus = copy.deepcopy(zrizy)
        elif tipo:
            universum = set(range(n))
            if tipo == "povne":
                self.R_plus = [set(range(n)) for _ in range(n)]
            elif tipo == "diag":
                self.R_plus = [{i} for i in range(n)]
            elif tipo == "antidiag":
                self.R_plus = [universum - {i} for i in range(n)]
            else:  # puste
                self.R_plus = [set() for _ in range(n)]
        else:
            self.R_plus = [set() for _ in range(n)]
    def peretyn(self, other):
        return VidnoshennyaZriz(self.n, [self.R_plus[i] & other.R_plus[i] for i in range(self.n)])
    def obyednannya(self, other):
        return VidnoshennyaZriz(self.n, [self.R_plus[i] | other.R_plus[i] for i in range(self.n)])
    def riznytsya(self, other):
        return VidnoshennyaZriz(self.n, [self.R_plus[i] - other.R_plus[i] for i in range(self.n)])
    def sym_riznytsya(self, other):
        return VidnoshennyaZriz(self.n, [self.R_plus[i] ^ other.R_plus[i] for i in range(self.n)])
    def dopovnennya(self):
        universum = set(range(self.n))
        return VidnoshennyaZriz(self.n, [universum - self.R_plus[i] for i in range(self.n)])
    def obernene(self):
        res = [set() for _ in range(self.n)]
        for x in range(self.n):
            for y in self.R_plus[x]:
                res[y].add(x)
        return VidnoshennyaZriz(self.n, res)
    def kompozytsiya(self, other):
        res = [set() for _ in range(self.n)]
        for i in range(self.n):
            for k in self.R_plus[i]:
                res[i] |= other.R_plus[k]
        return VidnoshennyaZriz(self.n, res)
    def zvuzhennya(self, subset: List[int]):
        subset_set = set(subset)
        return VidnoshennyaZriz(len(subset), [{y for y in self.R_plus[x] if y in subset_set} for x in subset])
    def vklyuchennya(self, other):
        for i in range(self.n):
            if not self.R_plus[i] <= other.R_plus[i]:
                return False
        return True
    def dvoiste(self):
        return self.to_matrix().dvoiste().to_zriz()
    def to_matrix(self):
        B = [[0]*self.n for _ in range(self.n)]
        for x in range(self.n):
            for y in sorted(self.R_plus[x]): 
                B[x][y] = 1
        return VidnoshennyaMatr(B)
    def print(self):
        for i, s in enumerate(self.R_plus):
            print(f"R⁺({i+1}) = {{{', '.join(str(x+1) for x in sorted(s))}}}")
        print()


P_matrix = [
    [1,0,1,1,0],
    [0,0,0,0,0],
    [0,0,1,1,1],
    [0,0,0,0,0],
    [0,0,1,0,0]
]

Q_matrix = [
    [1,1,0,0,0],
    [1,1,0,0,0],
    [0,0,1,1,1],
    [0,0,1,1,1],
    [0,0,1,1,1]
]

R_matrix = [
    [0,0,0,0,0],
    [1,0,0,0,0],
    [1,1,0,1,1],
    [1,0,0,0,0],
    [0,0,0,0,0]
]


P_m = VidnoshennyaMatr(P_matrix)
Q_m = VidnoshennyaMatr(Q_matrix)
R_m = VidnoshennyaMatr(R_matrix)

print("Результати для матричного представлення")
t1 = time.perf_counter()
PQ_m = P_m.kompozytsiya(Q_m)
t2 = time.perf_counter()
Rd_m = R_m.dvoiste()
t3 = time.perf_counter()
K_m = PQ_m.riznytsya(Rd_m)
t4 = time.perf_counter()

for row in K_m.B:
    print(row)
print(f"Час (композиція, обернене, різниця): ({t2-t1}, {t3-t2}, {t4-t3})")
print(f"Пам'ять: {sys.getsizeof(K_m.B)} байт\n")

P_z = P_m.to_zriz()
Q_z = Q_m.to_zriz()
R_z = R_m.to_zriz()

print("Результати для представлення зрізів")
t1 = time.perf_counter()
PQ_z = P_z.kompozytsiya(Q_z)
t2 = time.perf_counter()
Rd_z = R_z.dvoiste()
t3 = time.perf_counter()
K_z = PQ_z.riznytsya(Rd_z)
t4 = time.perf_counter()

for i, s in enumerate(K_z.R_plus):
    print(f"{i}: [{', '.join(str(x) for x in sorted(s))}]")
print(f"Час (композиція, обернене, різниця): ({t2-t1}, {t3-t2}, {t4-t3})")
print(f"Пам'ять: {sys.getsizeof(K_z.R_plus)} байт\n")

equal = (K_m.B == K_z.to_matrix().B)
print(f"Рівність результатів: {equal}")

Результати для матричного представлення
[0, 1, 1, 1, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 1, 1]
[0, 0, 0, 0, 0]
[0, 0, 1, 0, 0]
Час (композиція, обернене, різниця): (6.140000186860561e-05, 4.439998883754015e-05, 4.2400090023875237e-05)
Пам'ять: 120 байт

Результати для представлення зрізів
0: [1, 2, 3]
1: []
2: [3, 4]
3: []
4: [2]
Час (композиція, обернене, різниця): (5.789997521787882e-05, 8.590007200837135e-05, 5.149992648512125e-05)
Пам'ять: 120 байт

Рівність результатів: True
